# Puviyan Soil Detection - Enhanced Training v5.0

GPU-accelerated training with multiple dataset options:
- **Synthetic Data**: Generated soil-like images
- **Real Data**: Upload your own dataset
- **Hybrid**: Combination of both

**Setup:** Runtime > Change runtime type > GPU

In [ ]:
# Setup Environment
print('Setting up Puviyan Soil Detection Training v5.0...')
!pip install -q tensorflow matplotlib numpy tqdm pillow

import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from google.colab import files
import json
import os
import zipfile
from PIL import Image
import matplotlib.pyplot as plt

# Soil type labels
SOIL_LABELS = [
    "Alluvial Soil",
    "Black Soil", 
    "Red Soil",
    "Laterite Soil",
    "Desert Soil",
    "Saline/Alkaline Soil",
    "Peaty/Marshy Soil",
    "Forest/Hill Soil"
]

# Check GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    print(f'✅ GPU Available: {gpus[0]}')
    tf.config.experimental.set_memory_growth(gpus[0], True)
else:
    print('⚠️ No GPU detected - Training will be slow')

print(f'📊 Soil types: {len(SOIL_LABELS)}')
print('🚀 Environment ready!')

## 📥 Dataset Selection
Choose your training data source below:

In [ ]:
# Data loading functions
def generate_synthetic_data(num_samples=1000):
    """Generate synthetic soil-like images"""
    print(f'🎨 Generating {num_samples} synthetic soil samples...')
    
    # Generate base soil colors
    soil_colors = [
        [139, 69, 19],   # Brown (general soil)
        [160, 82, 45],   # Saddle brown
        [210, 180, 140], # Tan (sandy)
        [105, 105, 105], # Gray (clay)
        [139, 90, 43],   # Dark brown
        [222, 184, 135], # Burlywood
        [128, 128, 0],   # Olive (organic)
        [165, 42, 42]    # Brown (red soil)
    ]
    
    X = np.zeros((num_samples, 224, 224, 3), dtype=np.uint8)
    y = np.zeros(num_samples, dtype=np.int32)
    
    for i in range(num_samples):
        # Assign soil type
        soil_type = i % 8
        y[i] = soil_type
        
        # Base color
        base_color = soil_colors[soil_type]
        
        # Create image with base color
        img = np.full((224, 224, 3), base_color, dtype=np.uint8)
        
        # Add texture and variation
        noise = np.random.normal(0, 25, (224, 224, 3))
        img = np.clip(img + noise, 0, 255).astype(np.uint8)
        
        # Add some patterns
        if np.random.random() > 0.5:
            # Add spots/particles
            for _ in range(np.random.randint(5, 20)):
                x_pos = np.random.randint(0, 224)
                y_pos = np.random.randint(0, 224)
                size = np.random.randint(2, 8)
                color_var = np.random.randint(-50, 50, 3)
                spot_color = np.clip(np.array(base_color) + color_var, 0, 255)
                img[max(0, y_pos-size):min(224, y_pos+size), 
                    max(0, x_pos-size):min(224, x_pos+size)] = spot_color
        
        X[i] = img
    
    print(f'✅ Generated {num_samples} synthetic samples')
    return X, y

def load_real_data():
    """Load real soil images from uploaded zip file"""
    print('📤 Please upload your soil dataset zip file')
    print('Expected structure: soil_dataset.zip containing folders for each soil type')
    
    uploaded = files.upload()
    
    if not uploaded:
        print('⚠️ No files uploaded')
        return None, None
    
    zip_name = list(uploaded.keys())[0]
    print(f'📂 Extracting {zip_name}...')
    
    # Extract zip file
    with zipfile.ZipFile(zip_name, 'r') as zip_ref:
        zip_ref.extractall('dataset')
    
    # Load images
    X_list = []
    y_list = []
    
    dataset_path = 'dataset'
    
    # Find all image files
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                file_path = os.path.join(root, file)
                
                # Try to determine soil type from folder name
                folder_name = os.path.basename(root).lower()
                
                # Map folder names to soil types
                soil_type = 0  # Default to first type
                for i, label in enumerate(SOIL_LABELS):
                    if any(word in folder_name for word in label.lower().split()):
                        soil_type = i
                        break
                
                try:
                    # Load and resize image
                    img = Image.open(file_path)
                    img = img.convert('RGB')
                    img = img.resize((224, 224))
                    img_array = np.array(img)
                    
                    X_list.append(img_array)
                    y_list.append(soil_type)
                except Exception as e:
                    print(f'⚠️ Error loading {file_path}: {e}')
    
    if not X_list:
        print('❌ No valid images found')
        return None, None
    
    X = np.array(X_list)
    y = np.array(y_list)
    
    print(f'✅ Loaded {len(X)} real images')
    print(f'📊 Class distribution: {np.bincount(y)}')
    
    return X, y

print('📋 Data loading functions ready!')

In [ ]:
# Dataset selection
print('📊 Select your training dataset:')
print('1. Synthetic data only (fast, good for testing)')
print('2. Real data only (upload your dataset)')
print('3. Hybrid (synthetic + real data)')

choice = input('Enter your choice (1-3): ').strip()

X_train = None
y_train = None
X_val = None
y_val = None

if choice == '1':
    print('🎨 Using synthetic data only')
    X_all, y_all = generate_synthetic_data(5000)
    
elif choice == '2':
    print('📂 Using real data only')
    X_all, y_all = load_real_data()
    
    if X_all is None:
        print('⚠️ Falling back to synthetic data')
        X_all, y_all = generate_synthetic_data(5000)
        
elif choice == '3':
    print('🔄 Using hybrid data (synthetic + real)')
    
    # Generate synthetic data
    X_syn, y_syn = generate_synthetic_data(2500)
    
    # Load real data
    X_real, y_real = load_real_data()
    
    if X_real is not None:
        # Combine datasets
        X_all = np.concatenate([X_syn, X_real], axis=0)
        y_all = np.concatenate([y_syn, y_real], axis=0)
        print(f'✅ Combined dataset: {len(X_all)} samples')
    else:
        print('⚠️ Using synthetic data only')
        X_all, y_all = X_syn, y_syn
        
else:
    print('❌ Invalid choice, using synthetic data')
    X_all, y_all = generate_synthetic_data(5000)

# Split data
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(
    X_all, y_all, test_size=0.2, random_state=42, stratify=y_all
)

print(f'📊 Final dataset:')
print(f'   Training: {X_train.shape[0]} samples')
print(f'   Validation: {X_val.shape[0]} samples')
print(f'   Classes: {len(np.unique(y_all))}')

# Show sample images
plt.figure(figsize=(15, 3))
for i in range(8):
    plt.subplot(1, 8, i+1)
    # Find first sample of each class
    idx = np.where(y_train == i)[0]
    if len(idx) > 0:
        plt.imshow(X_train[idx[0]])
        plt.title(SOIL_LABELS[i], fontsize=8)
    plt.axis('off')
plt.tight_layout()
plt.show()

In [ ]:
# Create enhanced model
def create_enhanced_model():
    model = keras.Sequential([
        # Input preprocessing
        layers.Rescaling(1./255, input_shape=(224, 224, 3)),
        
        # Data augmentation (only during training)
        layers.RandomFlip('horizontal'),
        layers.RandomRotation(0.1),
        layers.RandomZoom(0.1),
        layers.RandomBrightness(0.1),
        
        # Feature extraction
        layers.Conv2D(32, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(64, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(128, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(2),
        
        layers.SeparableConv2D(256, 3, activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.GlobalAveragePooling2D(),
        
        # Classification head
        layers.Dense(512, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.3),
        layers.Dense(8, activation='softmax')
    ])
    
    return model

model = create_enhanced_model()
model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

print('🏗️ Enhanced model created!')
model.summary()

In [ ]:
# Train model
print('🚀 Starting training...')

# Callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=10,
        restore_best_weights=True,
        verbose=1
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=5,
        verbose=1
    )
]

# Train
history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

print('✅ Training completed!')

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Training')
plt.plot(history.history['val_accuracy'], label='Validation')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Training')
plt.plot(history.history['val_loss'], label='Validation')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

plt.tight_layout()
plt.show()

# Convert to TFLite
print('🔄 Converting to TFLite...')
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_model = converter.convert()

# Save TFLite model
model_filename = 'puviyan_soil_model.tflite'
with open(model_filename, 'wb') as f:
    f.write(tflite_model)

# Create labels JSON file for Flutter
labels_data = {
    "labels": SOIL_LABELS,
    "model_info": {
        "input_size": 224,
        "num_classes": 8,
        "model_type": "soil_classification",
        "version": "5.0",
        "dataset_type": choice
    }
}

labels_filename = 'labels.json'
with open(labels_filename, 'w') as f:
    json.dump(labels_data, f, indent=2)

print(f'✅ Model saved as {model_filename}')
print(f'📏 Model size: {len(tflite_model) / 1024:.1f} KB')
print(f'📋 Labels saved as {labels_filename}')

# Download both files
files.download(model_filename)
files.download(labels_filename)

print('🎉 Training complete! Both files downloaded successfully!')
print('Ready for Flutter integration!')